In [51]:
import spkit as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from copy import deepcopy

In [52]:
raw = np.array(pd.read_csv("K:\Clean Data\IGE\shirband\LB.csv", skiprows = 2))

In [53]:
X = deepcopy(raw[:, 2:])

In [54]:
ch_names = [str(i) for i in range(X.shape[1])]
sfreq = 300

In [55]:
info = mne.create_info(ch_names, sfreq, ch_types='eeg')
raw = mne.io.RawArray(X.T, info)

Creating RawArray with float64 data, n_channels=18, n_times=1815746
    Range : 0 ... 1815745 =      0.000 ...  6052.483 secs
Ready.


In [56]:
ica_low_cut = 1.0       # For ICA, we filter out more low-frequency power
hi_cut  = 120
raw_ica = raw.copy().filter(ica_low_cut, hi_cut)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 991 samples (3.303 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.0s finished


In [57]:
tstep = 1.0
events_ica = mne.make_fixed_length_events(raw_ica, duration=tstep)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=tstep,
                        baseline=None,
                        preload=True)

Not setting metadata
6052 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6052 events and 301 original time points ...
0 bad epochs dropped


In [58]:
from autoreject import get_rejection_threshold

reject = get_rejection_threshold(epochs_ica);
reject

Estimating rejection dictionary for eeg


{'eeg': 48.26591611253208}

In [60]:
# ICA parameters
random_state = 42   # ensures ICA is reproducable each time it's run
ica_n_components = .90     # Specify n_components as a decimal to set % explained variance

# Fit ICA
ica = mne.preprocessing.ICA(n_components=ica_n_components,
                            random_state=random_state,
                            )
ica.fit(epochs_ica,
        reject=reject,
        tstep=tstep)

Fitting ICA to data using 18 channels (please be patient, this may take a while)


C:\Users\Ozzy\AppData\Local\Temp\ipykernel_18348\2410709018.py:9: RuntimeWarning: The following parameters passed to ICA.fit() will be ignored, as they only affect raw data (and it appears you passed epochs): reject


Selecting by explained variance: 6 components
Fitting ICA took 12.2s.


Method,fastica
Fit,28 iterations on epochs (1821652 samples)
ICA components,6
Available PCA components,18
Channel types,eeg
ICA components marked for exclusion,—
